In [1]:
from typing import NamedTuple, Optional

# Define the context using NamedTuple


class Context(NamedTuple):
    id: int
    attributes: dict[str, Optional[str]]

    # Define the __hash__ method
    def __hash__(self) -> int:
        return hash(self.id)

    # Define the __eq__ method
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Context):
            return NotImplemented
        return self.id == other.id


# Define the Varible type
Variable = str

# Define the Assignment type
Assignment = dict[Variable, Context]

# Define the ContextSet type
ContextSet = set[Context]

In [2]:

# Define the base class for formula nodes
class FormulaNode:
    def evaluate(self, context: Context, assignment: Assignment) -> bool:
        raise NotImplementedError(
            "Evaluate method should be implemented by subclasses")

# Define NotNode class


class NotNode(FormulaNode):
    def __init__(self, node: FormulaNode):
        self.node = node

    def evaluate(self, context: Context, assignment: Assignment) -> bool:
        return not self.node.evaluate(context, assignment)

# Define AndNode class, which may have multiple children


class AndNode(FormulaNode):
    def __init__(self, *nodes: FormulaNode):
        if len(nodes) < 2:
            raise ValueError("AndNode should have at least 2 children")
        self.nodes = nodes

    def evaluate(self, context: Context, assignment: Assignment) -> bool:
        return all(node.evaluate(context, assignment) for node in self.nodes)

# Define OrNode class


class OrNode(FormulaNode):
    def __init__(self, *nodes: FormulaNode):
        if len(nodes) < 2:
            raise ValueError("OrNode should have at least 2 children")
        self.nodes = nodes

    def evaluate(self, context: Context, assignment: Assignment) -> bool:
        return any(node.evaluate(context, assignment) for node in self.nodes)

# Define ImpliesNode class


class ImpliesNode(FormulaNode):
    def __init__(self, left: FormulaNode, right: FormulaNode):
        self.left = left
        self.right = right

    def evaluate(self, context: Context, assignment: Assignment) -> bool:
        return not self.left.evaluate(context, assignment) or self.right.evaluate(context, assignment)

# Define EqualsNode class


class EqualsNode(FormulaNode):
    def __init__(self, var1: Variable, attr1: str, var2: Variable, attr2: str):
        self.var1 = var1
        self.var2 = var2
        self.attr1 = attr1
        self.attr2 = attr2

    def evaluate(self, context: Context, assignment: Assignment) -> bool:
        if self.var1 not in assignment or self.var2 not in assignment:
            raise ValueError("Variables not found in assignment")
        return assignment[self.var1].attributes[self.attr1] == assignment[self.var2].attributes[self.attr2]

# Define ForAllNode class


class ForAllNode(FormulaNode):
    def __init__(self, variable: Variable, node: FormulaNode, context_set: ContextSet):
        self.variable = variable
        self.node = node
        self.context_set = context_set

    def evaluate(self, context: Context, assignment: Assignment) -> bool:
        for context_id, context_attributes in self.context_set:
            new_assignment = assignment.copy()
            new_assignment[self.variable] = Context(
                context_id, context_attributes)
            if not self.node.evaluate(context, new_assignment):
                return False
        return True

# Define ExistsNode class


class ExistsNode(FormulaNode):
    def __init__(self, variable: Variable, node: FormulaNode, context_set: ContextSet):
        self.variable = variable
        self.node = node
        self.context_set = context_set

    def evaluate(self, context: Context, assignment: Assignment) -> bool:
        for context_id, context_attributes in self.context_set:
            new_assignment = assignment.copy()
            new_assignment[self.variable] = Context(
                context_id, context_attributes)
            if self.node.evaluate(context, new_assignment):
                return True
        return False

# Abstract Repair Action class


class RepairAction:
    def apply(self) -> None:
        raise NotImplementedError(
            "Apply method should be implemented by subclasses")

    def __str__(self) -> str:
        raise NotImplementedError(
            "__str__ method should be implemented by subclasses")

    def __eq__(self, other: object) -> bool:
        raise NotImplementedError(
            "__eq__ method should be implemented by subclasses")

    def __hash__(self) -> int:
        raise NotImplementedError(
            "__hash__ method should be implemented by subclasses")

# Define the AddAttributeAction class


class AddAttributeAction(RepairAction):
    def __init__(self, context: Context, attribute: str):
        self.context = context
        self.attribute = attribute

    def apply(self) -> None:
        self.context.attributes[self.attribute] = None

    def __str__(self) -> str:
        return f"{self.context.id}: +{self.attribute}"

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, AddAttributeAction):
            return NotImplemented
        return self.context == other.context and self.attribute == other.attribute

    def __hash__(self) -> int:
        return hash((self.context, self.attribute))

# Define the RemoveAttributeAction class


class RemoveAttributeAction(RepairAction):
    def __init__(self, context: Context, attribute: str):
        self.context = context
        self.attribute = attribute

    def apply(self) -> None:
        self.context.attributes.pop(self.attribute)

    def __str__(self) -> str:
        return f"{self.context.id}: -{self.attribute}"

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, RemoveAttributeAction):
            return NotImplemented
        return self.context == other.context and self.attribute == other.attribute

    def __hash__(self) -> int:
        return hash((self.context, self.attribute))

# Define the Repair Case class
# A repair case is a tuple of a repair action set and a weight


class RepairCase(NamedTuple):
    action: frozenset[RepairAction]
    weight: float

    # Define the __hash__ method
    def __hash__(self) -> int:
        return hash((self.action, self.weight))

    # Define the __eq__ method
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, RepairCase):
            return NotImplemented
        return self.action == other.action and self.weight == other.weight

    # Define the __str__ method
    def __str__(self) -> str:
        return f"{self.action}: {self.weight}"

    # Define the __and__ method
    def __and__(self, other: "RepairCase") -> "RepairCase":
        return RepairCase(self.action.union(other.action), self.weight + other.weight)

# Define the Repair Suite class
# A repair suite is a set of repair cases


class RepairSuite(set[RepairCase]):
    # Define the __str__ method
    def __str__(self) -> str:
        return "\n".join(str(repair_case) for repair_case in self)

    # Define the __or__ method
    def __or__(self, other: "RepairSuite") -> "RepairSuite":
        return RepairSuite(self.union(other))

    # Define the __and__ method
    def __and__(self, other: "RepairSuite") -> "RepairSuite":
        if self.isdisjoint(other):
            return self.union(other)
        return RepairSuite({repair_case1 & repair_case2 for repair_case1 in self for repair_case2 in other})

In [4]:
def example_usage():
    context1 = Context(1, {"attribute1": "value1", "attribute2": "value2"})
    context2 = Context(2, {"attribute1": "value2", "attribute2": "value2"})
    context3 = Context(3, {"attribute1": "value1", "attribute2": "value2"})
    context_set: ContextSet = {context1, context2, context3}

    equals_node = EqualsNode("x", "attribute1", "y", "attribute1")
    equals_node1 = EqualsNode("x", "attribute1", "x", "attribute1")
    equals_node2 = EqualsNode("y", "attribute2", "y", "attribute2")
    equals_node3 = EqualsNode("x", "attribute1", "y", "attribute2")

    and_node = AndNode(equals_node1, equals_node2, equals_node3)
    or_node = OrNode(equals_node1, equals_node2, equals_node3)
    implies_node = ImpliesNode(equals_node1, equals_node3)

    def test_formula_node(node: FormulaNode):
        assignment = {}
        forall_node = ForAllNode("x", ForAllNode(
            "y", node, context_set), context_set)
        exists_node = ExistsNode("x", ExistsNode(
            "y", node, context_set), context_set)
        print(f"Formula: {node}")
        print(f"ForAll: {forall_node.evaluate(context1, assignment)}")
        print(f"Exists: {exists_node.evaluate(context1, assignment)}")

    test_formula_node(equals_node)
    test_formula_node(and_node)
    test_formula_node(or_node)
    test_formula_node(implies_node)

In [5]:
example_usage()

Formula: <__main__.EqualsNode object at 0x0000013A4C728EC0>
ForAll: False
Exists: True
Formula: <__main__.AndNode object at 0x0000013A4C7291F0>
ForAll: False
Exists: True
Formula: <__main__.OrNode object at 0x0000013A4C729430>
ForAll: True
Exists: True
Formula: <__main__.ImpliesNode object at 0x0000013A4C7290D0>
ForAll: False
Exists: True
